In [ ]:
import requests

def type_id_query(type_id):
    # EVE Online API endpoint for fetching item details
    url = f"https://esi.evetech.net/latest/universe/types/{type_id}"

    # Make a GET request to the EVE Online API
    response = requests.get(url)

    # Check if the request was successful
    if response.status_code == 200:
        item_data = response.json()
        type_name = item_data.get("name", "Name not found")
        group_id = item_data.get("group_id", "Group ID not found")
        market_group_id = item_data.get("market_group_id", "Market Group ID not found")
        
        print(f"The name of the item with type_id {type_id} is: {type_name}")
        print(f"The group_id of the item with type_id {type_id} is: {group_id}")
        print(f"The market_group_id of the item with type_id {type_id} is: {market_group_id}")
        
        data_df = pd.DataFrame({"type_id":[type_id],"type_name": [type_name], "group_id": [group_id], "market_group_id": [market_group_id]})
        return data_df
    else:
        print(f"Failed to retrieve data. Status code: {response.status_code}")
        return None



In [ ]:
response = type_id_query(1215)

In [ ]:
def add_data_with_check(dbname, user, password, host, port, data, table_name):
    # Connect to the PostgreSQL database
    conn = psycopg2.connect(dbname=dbname, user=user, password=password, host=host, port=port)
    cur = conn.cursor()
    # Insert data
    print(data.head())
    for _, row in data.iterrows():
        try:
            cur.execute(sql.SQL("""
                INSERT INTO {} (date, region_id, type_id, average, highest, lowest, volume, order_count)
                VALUES (%s, %s, %s, %s, %s, %s, %s, %s);
            """).format(sql.Identifier(table_name)), 
                (row['date'], row['region_id'], row['type_id'], row['average'], row['highest'], row['lowest'], row['volume'], row['order_count']))
        except psycopg2.errors.ForeignKeyViolation as e:
            conn.rollback()
            print("Foreign key violation:", e)
            print("Attempting to fetch commodity using ESI API and add to commodites table...")
            esi_response = type_id_query(row['type_id'])
            
            if esi_response is not None:
                print("Commodity fetched successfully, adding to commodities table...")
                print("Commodity data:", esi_response)
                try:
                    cur.execute(sql.SQL("""
                        INSERT INTO commodities (type_id, group_id, type_name, market_group_id)
                        VALUES (%s, %s, %s, %s);
                    """), (esi_response['type_id'][0], esi_response['type_name'][0], esi_response['group_id'][0], esi_response['market_group_id'][0]))
                    print("Commodity added to commodities table successfully.")
                except psycopg2.errors.UniqueViolation as e:
                    print("Unique violation:", e)
                    print("Commodity already exists in commodities table, ignoring commodity...")
                    continue
            
            if esi_response is None:
                print("Failed to fetch commodity data from ESI API, ignoring commodity...")
                continue
            
    # Commit changes and close connection
    conn.commit()
    cur.close()
    conn.close()
            

In [ ]:
from esipy import EsiSecurity
from esipy import EsiClient

# basic client, for public endpoints only
# client = EsiClient(
#     retry_requests=True,  # set to retry on http 5xx error (default False)
#     headers={'User-Agent': 'Quintinius Orbin'},
#     raw_body_only=False,  # default False, set to True to never parse response and only return raw JSON string content.
# )
# creating the security object using the app
security = EsiSecurity(
    redirect_uri= config.redirect_uri,
    headers={'User-Agent': 'Quintinius Orbin'},
    client_id=config.client_id,
    secret_key=config.secret_key,
)

# when you have a security object, you need to give it to the client
# so he knows where to get auth headers for authed endpoints.
# --> simplified client
client = EsiClient(security=security, 
                   headers={'User-Agent': 'Quintinius Orbin'}
                   )